## DP for Paths in a Matrix
* these problems have matrices as part of the input and give rules for moving through the matrix in the problem description
* typically, DP will be applcable when the allowed movement is constrained in a way that prevent moving backwars. For example, if we are only allowed to move down and right
  + if we are allowed to move in all 4 directions, then it might be a graph/BFS problem instead
* this patten sometimes is combined with other patterns such as counting DP  

### Leetcode 62 Unique Paths
* overview
  + There is a robot on an m x n grid. The robot is initially located at the top-left corner (i.e., grid\[0\]\[0\]). The robot tries to move to the bottom-right corner (i.e., grid\[m - 1\]\[n - 1\]). The robot can only move either down or right at any point in time
  + Given the two integers m and n, return the number of possible unique paths that the robot can take to reach the bottom-right corner
* algorithm
  + state variables: this is usually straightforward in path problems. We use a two dimensional matrix to record the positions on the matrix, and the values of the matrix elements correspond to the number of paths
  + recurrence relation
    + this is also straightforward, for each cell, the value equals to the sum of the cells with the possible directions leading to the current cell. dp(row)(col) = dp(row-1)(col) + dp(row)(col-1) if row-1, or col-1 are not out of boundary
  + base cases
    + at the starting position, there is only one way to get there, therefore, we have dp(0)(0) = 1
  + optimization on space complexity
    + we only need the same column of the previous row, and the left cell on the same row
    + in addition, we can exchange the row and columns to operate on the smaller one, and the same recurrence relationship applies: we need the same index (i) element of previous dp array, and the i-1 element of the current array to derive the current value at i

In [1]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        if m * n == 0:
            return 0
        
        (m, n) = (n, m) if m < n else (m, n)
        
        pre = [1] * n
        
        for _ in range(1, m):
            curr = [0] * n
            curr[0] = 1
            for i in range(1, n):
                curr[i] += curr[i-1] + pre[i]
            pre = curr
            
        return pre[n-1]    
        

### Leetcode 63 Unique Paths II
* overview
  + You are given an m x n integer array grid. There is a robot initially located at the top-left corner (i.e., grid\[0\]\[0\]). The robot tries to move to the bottom-right corner (i.e., grid\[m - 1\]\[n - 1\]). The robot can only move either down or right at any point in time.
  + An obstacle and space are marked as 1 or 0 respectively in grid. A path that the robot takes cannot include any square that is an obstacle.
  + Return the number of possible unique paths that the robot can take to reach the bottom-right corner.
* algorithm
  + this is the extension to Leetcode 62, unique path, and added the restricution of obstacles
  + we use the similar algorithm, but only calculate the number of ways when obstacleGrid elements have a value not equal to 1
  + in addition, when initializing pre and curr, we check if the corresponding element in obstacleGrid is 1, if so, just initialize it as 0, otherwise, we apply the transfer rules either from its left or top
    + for pre array, we only consider its left element to transfer
    + for curr array, we only consider its top element, which is the pre\[j\] to transfer

In [3]:
from typing import List
class Solution:
    def uniquePathsWithObstacles(self, obstacleGrid: List[List[int]]) -> int:
        if not obstacleGrid or not obstacleGrid[0] or obstacleGrid[0][0] == 1:
            return 0
        
        m, n = len(obstacleGrid), len(obstacleGrid[0])        
        
        pre = [0] * n
        pre[0] = 1
        for k in range(1, n):
            if obstacleGrid[0][k]!= 1 and pre[k-1] != 0:
                pre[k] = 1
        
        for i in range(1, m):
            curr = [0] * n
            curr[0] = pre[0] if obstacleGrid[i][0] != 1 else 0
            for j in range(1, n):
                if obstacleGrid[i][j] != 1:
                    curr[j] += curr[j-1] + pre[j]
            pre = curr
        return pre[n-1]    
        

### Leetcode 64 Minimum Path Sum
* overview
  + Given a m x n grid filled with non-negative numbers, find a path from top left to bottom right, which minimizes the sum of all numbers along its path.
  + Note: You can only move either down or right at any point in time.
* algorithm
  + this is the extension to the Leetcode 62 Unique Paths, we apply a similar algorithm
  + when initializing the pre array, we copy the first row from grid matrix, and get the cumsum of this row, which is the cost to move from the top left to the position of each cell
  + we then traverse from row 1 to m-1
    + first initialize curr array, and for its first element, update its value from the top path, which is the only path for the left-most column
    + then we traverse from column indexed 1 to n-1, to update the curr array by both top and left paths, plus the value from the corresponding cell of the Grid matrix
    + we assign curr to prev at the end of the iner for loop
 + return the n-1th element of either curr or pre array 
* time complexity O(mn)
* space complexity O(n) after optimization

In [ ]:
class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        if not grid or not grid[0]:
            return 0
        
        m, n = len(grid), len(grid[0])
        
        pre = grid[0].copy()
        
        # the first row only has one path from its left
        for i in range(1, n):
            pre[i] += pre[i-1] 
            
        for i in range(1, m):
            curr = [0] * n
            
            # the left most column only has one path from its top
            curr[0] = pre[0] + grid[i][0]
            
            # the internal cells will have the min path from the top and left
            for j in range(1, n):
                curr[j] = min(pre[j], curr[j-1]) + grid[i][j]
                
            pre = curr
        return pre[n-1]    

### Leetcode 931 Minimum Falling Path Sum
* overview
  + Given an n x n array of integers matrix, return the minimum sum of any falling path through matrix
  + A falling path starts at any element in the first row and chooses the element in the next row that is either directly below or diagonally left/right. Specifically, the next element from position (row, col) will be (row + 1, col - 1), (row + 1, col), or (row + 1, col + 1).
* algorithm
  + we build an space optimized algorithm by starting from the last row to first row, and return the min of the first row
  + we initialize a pre array with n+1 elements, the last one represent the imaginary inf element for the right-most matrix element
  + for the left-most element, we initialize a pre = inf for it to select
  + traverse from the last row to the first row, each iteration, we set pre=inf, then traverse the row
    + we obtain the min_add by the min of pre, dp(j), dp(j+1)
    + we then set pre=dp(j), before we update dp(j)
    + then set dp(j) = matrix(i)(j) + min_add
  + the key point is that whenever we update dp(j), since dp(j) will be used for the next j as dp(j-1), so we can't use dp(j-1) because the value has been updated to the next iteration's value, so we just assign dp(j) to pre before we update its value. Then when calculating add, we use min(pre, dp(j), dp(j+1)). we don't need to worry about dp(j) and dp(j+1) since we haven't updated their values yet when we calcualte dp(j) value, but we have updated the dp(j-1) value  

In [ ]:
class Solution:
    def minFallingPathSum(self, matrix: List[List[int]]) -> int:
        if not matrix or not matrix[0]:
            return 0
        
        m, n = len(matrix), len(matrix[0])
        
        dp = matrix[m-1].copy()
        dp.append(float("inf"))
                
        for i in range(m-2, -1, -1):
            pre = float("inf")            
            for j in range(n):
                add = min(pre, dp[j], dp[j+1])
                pre = dp[j]
                dp[j] = add + matrix[i][j]                
        
        return min(dp)            
        